In [1]:
import ee
import geemap.core as geemap
import datetime

In [2]:
# Authenticate and initialize Earth Engine

ee.Authenticate()
ee.Initialize(project='tz-sgr')


## Define Tanzania boundary

In [3]:

# Load country boundaries
tanzania = (
    ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
    .filter(ee.Filter.eq('country_na', 'Tanzania'))
)

# Geometry only (important later)
tz_geom = tanzania.geometry()


In [4]:
# Grid: use grid previously uploaded to Earth Engine assets
grid = ee.FeatureCollection('users/franziskajzollner/tanzania_1km_grid')

print('First 3 features:', grid.limit(3).getInfo())


First 3 features: {'type': 'FeatureCollection', 'columns': {'grid_id': 'String', 'system:index': 'String'}, 'version': 1769090688631461, 'id': 'users/franziskajzollner/tanzania_1km_grid', 'properties': {'system:asset_size': 201679338}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[34.49956008332153, -2.0022667746310043], [34.50852641182152, -2.0022915609347627], [34.508550930444926, -1.9932703988171274], [34.49958468900799, -1.9932457578374796], [34.49956008332153, -2.0022667746310043]]]}, 'id': '0000000000000000518b', 'properties': {'grid_id': '34.0_-2.0'}}, {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[34.50852641182152, -2.0022915609347627], [34.51749275309369, -2.002316310087368], [34.5175172692081, -1.993295001057934], [34.508550930444926, -1.9932703988171274], [34.50852641182152, -2.0022915609347627]]]}, 'id': '0000000000000000518c', 'properties': {'grid_id': '34.0_-2.0'}}, {'type': 'Feature', 'geometry': {'type': 'Poly

## Nighttime Lights data (NASA Black Marble)

In [5]:
vnp = (ee.ImageCollection('NASA/VIIRS/002/VNP46A2')
       .select('Gap_Filled_DNB_BRDF_Corrected_NTL'))


## Create monthly means & reduce each month over the grid


In [11]:
# Define a function to compute monthly mean nightlights for a given year

def monthly_mean(year):
    year = ee.Number(year)
    months = ee.List.sequence(1, 2) #original code: "sequence(1, 12)" to cover all months

    def month_img(m):
        m = ee.Number(m)
        start = ee.Date.fromYMD(year, m, 1)
        end = start.advance(1, 'month')

        img = (vnp
               .filterDate(start, end)
               .filterBounds(tz_geom)
               .mean()
               .clip(tz_geom))

        # Name the band with year-month, e.g. NTL_2020_01
        band_name = ee.String('NTL_').cat(year.format()).cat('_').cat(m.format('%02d'))
        img = img.rename(band_name)

        # Carry metadata to identify the month if you need it later
        return img.set({
            'year': year,
            'month': m,
            'band_name': band_name
        })

    return ee.ImageCollection(months.map(month_img))


# Loop over years, reduce data over grid and export monthly means per grid cell

for year in range(2025, 2026): #original code: "range(2023, 2026)" since train was opened in 2024, so I want to check pre- and post-period
    print(f"Processing year {year}")
    monthly_ic = monthly_mean(year)

    # Get actual number of months (safe)
    band_names = monthly_ic.aggregate_array('band_name').getInfo()
    print(f"Found months: {band_names}")

    for band_name in band_names:
        month_img = monthly_ic.filter(ee.Filter.eq('band_name', band_name)).first()
        
        stats = month_img.reduceRegions(grid, ee.Reducer.mean(), 500)
        
        # Rename mean → proper column name
        stats_final = stats.map(
            lambda f: f.set(band_name, f.get('mean'))
        )
        
        
        ee.batch.Export.table.toDrive(
            collection=stats_final,
            description=f'tz_ntl_{band_name.replace("NTL_", "")}', 
            fileFormat='CSV'
        ).start()


Processing year 2025
Found months: ['NTL_2025_01', 'NTL_2025_02']


In [1]:
# open CSV file to explore
from pandas import read_csv

nightlights_data = read_csv('C:\\Users\\zollner\\OneDrive - epfl.ch\\Documents\\SGR TZ\\Data\\Processed\\tz_ntl_2025_01.csv') # Example for January 2025


## Problems: 

1. Grid-ID is not unique
2. Mean NL data is in there twice